In [1]:
# import modules
from datasets import load_dataset 
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BertForSequenceClassification
import torch
from tqdm import tqdm
import pickle
from torch.utils.data import DataLoader, Sampler, BatchSampler
from torch.nn.utils import parameters_to_vector, vector_to_parameters
from torch import nn
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
import random

ideas
- show that gradients become more similar after gram Schmidt orthogonalization

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# load model and tokenizer
model_orig = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', device_map="cuda")
model_diff = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', device_map="cuda")
#model_gib = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
dataset = load_dataset(
    "json",
    data_files={
      "train": "data/mrpc_gibberish_train_filtered.json",
      "test":  "data/mrpc_gibberish_test_filtered.json"
    },
    field=None
)

# Define an encoding function that preserves the gibberish fields.
def encode(examples):
    tokenized = tokenizer(
        examples["sentence1"],
        examples["sentence2"],
        truncation=True,
        padding="max_length"
    )
    # Ensure we preserve the original gibberish fields.
    tokenized["sentence1_gibberish"] = examples["sentence1_gibberish"]
    tokenized["sentence2_gibberish"] = examples["sentence2_gibberish"]
    return tokenized

# Apply encoding with batched=True (this will preserve keys if you don't remove them).
dataset = dataset.map(encode, batched=True)

# When mapping the labels, explicitly not remove any columns.
dataset = dataset.map(lambda examples: {"labels": examples["label"]}, batched=True, remove_columns=[])

In [5]:
dataset['train'].set_format(
    type='torch', 
    columns=[
        'input_ids', 'token_type_ids', 'attention_mask', 'labels',
        'sentence1_gibberish', 'sentence2_gibberish'
    ]
)
dataset['test'].set_format(
    type='torch', 
    columns=[
        'input_ids', 'token_type_ids', 'attention_mask', 'labels'
    ]
)

In [6]:
def gram_schmidt(v_gib, v_info):
    len_orig = torch.pow(torch.dot(v_info, v_info), .5)
    len_gib = torch.pow(torch.dot(v_gib, v_gib), .5)
    projection = (torch.dot(v_info, v_gib) / torch.dot(v_gib, v_gib)) * v_gib
    u2 = v_info - projection
    u3 = u2 * (len_orig / len_gib)
    return u3

In [7]:
class BalancedBatchSampler(Sampler):
    def __init__(self, labels, batch_size):
        assert batch_size % 2 == 0, "Batch size must be even for two classes"
        self.labels = labels
        self.bs = batch_size
        # split indices by class
        self.class_indices = {
            0: [i for i, lab in enumerate(labels) if lab == 0],
            1: [i for i, lab in enumerate(labels) if lab == 1],
        }
        self.num_batches = min(len(self.class_indices[0]), len(self.class_indices[1])) * 2 // batch_size

    def __iter__(self):
        # shuffle each class’s indices
        for idx_list in self.class_indices.values():
            random.shuffle(idx_list)
        # yield batch_count batches
        for i in range(self.num_batches):
            half = self.bs // 2
            # take a slice from each class
            start = i * half
            batch = (
                self.class_indices[0][start:start+half] +
                self.class_indices[1][start:start+half]
            )
            random.shuffle(batch)
            yield batch

    def __len__(self):
        return self.num_batches

# usage:
batch_size = 4 # batch size of 6 is already taking up too much VRAM
labels = dataset["train"]["labels"]
balanced_sampler = BalancedBatchSampler(labels, batch_size=batch_size)


In [8]:
def my_collate(batch):
    return {
        'input_ids':      torch.stack([item['input_ids']      for item in batch]), # token IDs like: [ CLS ]  tokens_of_sentence1  [ SEP ]  tokens_of_sentence2  [ SEP ]  padding… 
        'token_type_ids': torch.stack([item['token_type_ids'] for item in batch]), # whether the token belonged to sentence1 or sentence2
        'attention_mask': torch.stack([item['attention_mask'] for item in batch]), # marks which positions are padding vs. real tokens
        'labels':         torch.tensor([item['labels']         for item in batch]), # prediction labels (0, 1)
        'sentence1_gibberish': [item.get('sentence1_gibberish', []) for item in batch],
        'sentence2_gibberish': [item.get('sentence2_gibberish', []) for item in batch],
    }

# 4) Prepare DataLoaders
train_loader = DataLoader(
    dataset['train'], batch_sampler=balanced_sampler, collate_fn=my_collate
)
""" batch_size = 4
train_loader = DataLoader(
    dataset['train'], batch_size=batch_size, shuffle=True, collate_fn=my_collate
) """

test_loader = DataLoader(
    dataset['test'], batch_size=batch_size, shuffle=False,
    collate_fn=lambda b: {
        'input_ids': torch.stack([x['input_ids'] for x in b]),
        'token_type_ids': torch.stack([x['token_type_ids'] for x in b]),
        'attention_mask': torch.stack([x['attention_mask'] for x in b]),
        'labels': torch.tensor([x['labels'] for x in b]),
    }
)

In [9]:
optimizer_orig = torch.optim.AdamW(model_orig.parameters(), lr=1e-5)
optimizer_diff = torch.optim.AdamW(model_diff.parameters(), lr=1e-5)

check for potential errors in the logic: e.g. 
- is the model updated on the correct gradient vector?
- is the Gram-Schmidt process calculated correctly?

In [10]:
""" # 6) Training loop on train_loader
num_epochs = 3
for epoch in range(num_epochs):
    model_orig.train(); model_diff.train()
    for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}"), start=1):
        for k, v in batch.items(): # extracts keys (e.g. 'input_ids') and values (actual tokens) from batch
            if torch.is_tensor(v): batch[k] = v.to(device) # moves value to GPU, if it is a token

        # unpack
        input_ids = batch['input_ids']
        token_type_ids = batch['token_type_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        gib1_all = batch['sentence1_gibberish']
        gib2_all = batch['sentence2_gibberish']
        num_variants = len(gib1_all[0])

        # Pre-tokenize gibberish variants
        gib_inputs = []
        for j in range(num_variants):
            s1 = [g[j] for g in gib1_all]
            s2 = [g[j] for g in gib2_all]
            enc = tokenizer(s1, s2, truncation=True, padding='max_length', return_tensors='pt')
            gib_inputs.append({k: v.to(device) for k, v in enc.items()})

        # A) Original pass
        optimizer_orig.zero_grad()
        out_orig = model_orig(input_ids=input_ids,
                              token_type_ids=token_type_ids,
                              attention_mask=attention_mask,
                              labels=labels)
        loss_orig = out_orig.loss
        loss_orig.backward()
        optimizer_orig.step()
        orig_grads = [p.grad.detach().cpu().clone() for p in model_orig.parameters()]

        # B) Gibberish pass
        optimizer_orig.zero_grad()
        gib_loss = torch.tensor(0.0, device=device)
        for enc in gib_inputs:
            out = model_orig(**enc, labels=labels)
            gib_loss += out.loss
        gib_loss /= num_variants
        gib_loss.backward()
        gib_grads = [p.grad.detach().cpu().clone() for p in model_orig.parameters()]

        # C) Compute orthogonal vector
        orig_vec = parameters_to_vector(orig_grads)
        gib_vec = parameters_to_vector(gib_grads)
        v_orth = gram_schmidt(gib_vec, orig_vec)
        #v_diff = orig_vec - orig_vec

        # D) Update model_diff

        optimizer_diff.zero_grad() # clear old grads
        # unflatten v_diff → p.grad for each parameter p
        pointer = 0
        for p in model_diff.parameters():
            if not p.requires_grad:
                continue
            numel = p.numel()
            p.grad = v_orth[pointer:pointer+numel].view_as(p).to(device)
            pointer += numel
        optimizer_diff.step()


        if batch_idx % 5 == 0:
            print(f"[Epoch {epoch+1} | Batch {batch_idx}] "
                  f"orig_loss = {loss_orig.item():.4f}, gib_loss = {gib_loss.item():.4f}")


    print(f"Epoch {epoch+1} complete")

model_orig.save_pretrained("trained_model_original")
model_diff.save_pretrained("trained_model_gradient_diff") """

' # 6) Training loop on train_loader\nnum_epochs = 3\nfor epoch in range(num_epochs):\n    model_orig.train(); model_diff.train()\n    for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}"), start=1):\n        for k, v in batch.items(): # extracts keys (e.g. \'input_ids\') and values (actual tokens) from batch\n            if torch.is_tensor(v): batch[k] = v.to(device) # moves value to GPU, if it is a token\n\n        # unpack\n        input_ids = batch[\'input_ids\']\n        token_type_ids = batch[\'token_type_ids\']\n        attention_mask = batch[\'attention_mask\']\n        labels = batch[\'labels\']\n        gib1_all = batch[\'sentence1_gibberish\']\n        gib2_all = batch[\'sentence2_gibberish\']\n        num_variants = len(gib1_all[0])\n\n        # Pre-tokenize gibberish variants\n        gib_inputs = []\n        for j in range(num_variants):\n            s1 = [g[j] for g in gib1_all]\n            s2 = [g[j] for g in gib2_all]\n            enc = tokeniz

note: model seems to be 100% sure, that every pair is not the same

In [11]:
# 6) Training loop on train_loader
num_epochs = 3
for epoch in range(num_epochs):
    model_orig.train(); model_diff.train()
    for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}"), start=1):
        for k, v in batch.items(): # extracts keys (e.g. 'input_ids') and values (actual tokens) from batch
            if torch.is_tensor(v): batch[k] = v.to(device) # moves value to GPU, if it is a token

        # unpack
        input_ids = batch['input_ids']
        token_type_ids = batch['token_type_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        gib1_all = batch['sentence1_gibberish']
        gib2_all = batch['sentence2_gibberish']
        num_variants = len(gib1_all[0])

        # Pre-tokenize gibberish variants
        gib_inputs = []
        for j in range(num_variants):
            s1 = [g[j] for g in gib1_all]
            s2 = [g[j] for g in gib2_all]
            enc = tokenizer(s1, s2, truncation=True, padding='max_length', return_tensors='pt')
            gib_inputs.append({k: v.to(device) for k, v in enc.items()})

        # A) Original pass
        optimizer_orig.zero_grad()
        out_orig = model_orig(input_ids=input_ids,
                              token_type_ids=token_type_ids,
                              attention_mask=attention_mask,
                              labels=labels)
        loss_orig = out_orig.loss
        loss_orig.backward()
        optimizer_orig.step()
        #orig_grads = [p.grad.detach().cpu().clone() for p in model_orig.parameters()]


        # B) Gibberish pass
        # B.1 pass for original data
        optimizer_diff.zero_grad()
        out_orig_model_diff = model_diff(input_ids=input_ids,
                                            token_type_ids=token_type_ids,
                                            attention_mask=attention_mask,
                                            labels=labels)
        loss_orig_model_diff = out_orig_model_diff.loss
        loss_orig_model_diff.backward()
        orig_grads_model_diff = [p.grad.detach().cpu().clone() for p in model_diff.parameters()]

        # B.2 pass for gibberish data
        optimizer_diff.zero_grad()
        gib_loss = torch.tensor(0.0, device=device)
        for enc in gib_inputs:
            out = model_diff(**enc, labels=labels)
            gib_loss += out.loss
        gib_loss /= num_variants
        gib_loss.backward()
        gib_grads = [p.grad.detach().cpu().clone() for p in model_diff.parameters()]

        # C) Compute orthogonal vector
        orig_vec_model_diff = parameters_to_vector(orig_grads_model_diff)
        gib_vec = parameters_to_vector(gib_grads)
        v_orth = gram_schmidt(gib_vec, orig_vec_model_diff)
        #v_diff = orig_vec - orig_vec

        # D) Update model_diff
        optimizer_diff.zero_grad() # clear old grads
        # unflatten v_orth → p.grad for each parameter p
        pointer = 0
        for p in model_diff.parameters():
            if not p.requires_grad:
                continue
            numel = p.numel()
            p.grad = v_orth[pointer:pointer+numel].view_as(p).to(device)
            pointer += numel
        optimizer_diff.step()


        if batch_idx % 5 == 0:
            print(f"[Epoch {epoch+1} | Batch {batch_idx}] "
                  f"orig_loss = {loss_orig.item():.4f}, gib_loss = {gib_loss.item():.4f}, labels: {labels[:]}")


    print(f"Epoch {epoch+1} complete")

model_orig.save_pretrained("trained_model_original_2")
model_diff.save_pretrained("trained_model_gradient_orth_2")

Epoch 1:   5%|▍         | 5/106 [00:27<08:58,  5.33s/it]

[Epoch 1 | Batch 5] orig_loss = 0.6323, gib_loss = 0.7383, labels: tensor([0, 1, 0, 1], device='cuda:0')


Epoch 1:   9%|▉         | 10/106 [00:51<07:38,  4.78s/it]

[Epoch 1 | Batch 10] orig_loss = 0.8053, gib_loss = 0.7138, labels: tensor([0, 0, 1, 1], device='cuda:0')


Epoch 1:  14%|█▍        | 15/106 [01:15<07:10,  4.73s/it]

...

[Epoch 3 | Batch 100] orig_loss = 0.3919, gib_loss = 0.6420, labels: tensor([0, 1, 1, 0], device='cuda:0')


Epoch 3:  99%|█████████▉| 105/106 [08:43<00:05,  5.03s/it]

[Epoch 3 | Batch 105] orig_loss = 0.3217, gib_loss = 0.5815, labels: tensor([0, 1, 1, 0], device='cuda:0')


Epoch 3: 100%|██████████| 106/106 [08:48<00:00,  4.99s/it]


Epoch 3 complete


In [12]:
# 7) Evaluation on test_loader
def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            logits = model(
                input_ids=batch['input_ids'],
                token_type_ids=batch['token_type_ids'],
                attention_mask=batch['attention_mask']
            ).logits
            preds = torch.argmax(logits, dim=-1)
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(batch['labels'].cpu().tolist())
    return all_labels, all_preds

model_orig = BertForSequenceClassification.from_pretrained("./trained_model_original")
model_orig.to(device)

model_diff = BertForSequenceClassification.from_pretrained("./trained_model_gradient_diff")
model_diff.to(device)

# Compute metrics for each model
labels_o, preds_o = evaluate(model_orig, test_loader)
labels_d, preds_d = evaluate(model_diff, test_loader)

print("Original Model Metrics:")
print(classification_report(labels_o, preds_o, digits=4))

print("Orthogonal trained Model Metrics:")
print(classification_report(labels_d, preds_d, digits=4))

Original Model Metrics:
              precision    recall  f1-score   support

           0     0.8000    0.3448    0.4819        58
           1     0.7286    0.9533    0.8259       107

    accuracy                         0.7394       165
   macro avg     0.7643    0.6490    0.6539       165
weighted avg     0.7537    0.7394    0.7050       165

Orthogonal trained Model Metrics:
              precision    recall  f1-score   support

           0     0.3515    1.0000    0.5202        58
           1     0.0000    0.0000    0.0000       107

    accuracy                         0.3515       165
   macro avg     0.1758    0.5000    0.2601       165
weighted avg     0.1236    0.3515    0.1829       165



/home/tschuetz/.conda/envs/tilo_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/tschuetz/.conda/envs/tilo_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/tschuetz/.conda/envs/tilo_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

thoughts: the models learns at least in some direction, as it always predicts the minority class. Usually, it is not incentivized to do that

In [13]:
# 6) Training loop on train_loader
num_epochs = 3
for epoch in range(num_epochs):
    model_orig.train(); model_diff.train()
    for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}"), start=1):
        for k, v in batch.items(): # extracts keys (e.g. 'input_ids') and values (actual tokens) from batch
            if torch.is_tensor(v): batch[k] = v.to(device) # moves value to GPU, if it is a token

        # unpack
        input_ids = batch['input_ids']
        token_type_ids = batch['token_type_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        gib1_all = batch['sentence1_gibberish']
        gib2_all = batch['sentence2_gibberish']
        num_variants = len(gib1_all[0])

        # Pre-tokenize gibberish variants
        gib_inputs = []
        for j in range(num_variants):
            s1 = [g[j] for g in gib1_all]
            s2 = [g[j] for g in gib2_all]
            enc = tokenizer(s1, s2, truncation=True, padding='max_length', return_tensors='pt')
            gib_inputs.append({k: v.to(device) for k, v in enc.items()})

        # A) Original pass
        optimizer_orig.zero_grad()
        out_orig = model_orig(input_ids=input_ids,
                              token_type_ids=token_type_ids,
                              attention_mask=attention_mask,
                              labels=labels)
        loss_orig = out_orig.loss
        loss_orig.backward()
        optimizer_orig.step()
        #orig_grads = [p.grad.detach().cpu().clone() for p in model_orig.parameters()]


        # B) Gibberish pass
        # B.1 pass for original data
        optimizer_diff.zero_grad()
        out_orig_model_diff = model_diff(input_ids=input_ids,
                                            token_type_ids=token_type_ids,
                                            attention_mask=attention_mask,
                                            labels=labels)
        loss_orig_model_diff = out_orig_model_diff.loss
        loss_orig_model_diff.backward()
        orig_grads_model_diff = [p.grad.detach().cpu().clone() for p in model_diff.parameters()]

        # B.2 pass for gibberish data
        optimizer_diff.zero_grad()
        gib_loss = torch.tensor(0.0, device=device)
        for enc in gib_inputs:
            out = model_diff(**enc, labels=labels)
            gib_loss += out.loss
        gib_loss /= num_variants
        gib_loss.backward()
        gib_grads = [p.grad.detach().cpu().clone() for p in model_diff.parameters()]

        # C) Compute orthogonal vector
        orig_vec_model_diff = parameters_to_vector(orig_grads_model_diff)
        gib_vec = parameters_to_vector(gib_grads)
        v_orth = gram_schmidt(gib_vec, orig_vec_model_diff)
        #v_diff = orig_vec - orig_vec

        # D) Update model_diff
        optimizer_diff.zero_grad() # clear old grads
        # unflatten v_orth → p.grad for each parameter p
        pointer = 0
        for p in model_diff.parameters():
            if not p.requires_grad:
                continue
            numel = p.numel()
            p.grad = v_orth[pointer:pointer+numel].view_as(p).to(device)
            pointer += numel
        optimizer_diff.step()


        if batch_idx % 5 == 0:
            print(f"[Epoch {epoch+1} | Batch {batch_idx}] "
                  f"orig_loss = {loss_orig.item():.4f}, gib_loss = {gib_loss.item():.4f}")


    print(f"Epoch {epoch+1} complete")

model_orig.save_pretrained("trained_model_original_2")
model_diff.save_pretrained("trained_model_gradient_orth_2")

Epoch 1:   5%|▍         | 5/106 [00:24<08:09,  4.84s/it]

[Epoch 1 | Batch 5] orig_loss = 0.6867, gib_loss = 0.8929


Epoch 1:   9%|▉         | 10/106 [00:53<08:34,  5.35s/it]

[Epoch 1 | Batch 10] orig_loss = 0.0779, gib_loss = 0.8759


Epoch 1:  14%|█▍        | 15/106 [01:17<07:26,  4.90s/it]

...

[Epoch 3 | Batch 100] orig_loss = 0.0612, gib_loss = 0.8719


Epoch 3:  99%|█████████▉| 105/106 [09:04<00:05,  5.26s/it]

[Epoch 3 | Batch 105] orig_loss = 0.6191, gib_loss = 0.9001


Epoch 3: 100%|██████████| 106/106 [09:09<00:00,  5.18s/it]


Epoch 3 complete
